In [1]:
import finetune_wav2vec2_utils as w2v2
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms
from torch.utils.data import DataLoader
import numpy as np
import os

/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-23 02:56:28.819963: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 02:56:28.827656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-23 02:56:28.838717: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-0

In [2]:
from transformers import Wav2Vec2Model, Wav2Vec2Processor
import pandas as pd

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
# Initialize processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [4]:
from sklearn.model_selection import train_test_split

In [5]:
csv_file = './dataClassified/data.csv'
df = pd.read_csv(csv_file)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, 
                                       stratify=df['label'])

In [6]:
train_df.get('file_path').values

array(['scream_data/kaggle_scream_740.wav',
       'control_data/libri_control_1931.wav',
       'control_data/kaggle_control_2035.wav', ...,
       'scream_data/kaggle_scream_238.wav',
       'control_data/libri_control_1122.wav',
       'control_data/libri_control_923.wav'], dtype=object)

In [7]:
# Save splits to temporary CSVs
train_df.to_csv('train_temp.csv', index=False)
val_df.to_csv('val_temp.csv', index=False)

In [8]:
train_dataset = w2v2.AudioDataset('train_temp.csv', processor)
val_dataset = w2v2.AudioDataset('val_temp.csv', processor)

In [9]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)
    

In [10]:
train_dataset[0]

(tensor([[-3.3342e+01,  1.5089e+01,  8.6710e+01,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11],
         [ 2.2119e-11,  2.2119e-11,  2.2119e-11,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11],
         [ 2.2119e-11,  2.2119e-11,  2.2119e-11,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11],
         ...,
         [ 2.2119e-11,  2.2119e-11,  2.2119e-11,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11],
         [ 2.2119e-11,  2.2119e-11,  2.2119e-11,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11],
         [ 2.2119e-11,  2.2119e-11,  2.2119e-11,  ...,  2.2119e-11,
           2.2119e-11,  2.2119e-11]]),
 tensor(0))

In [11]:
model = w2v2.AudioClassifier(num_classes=3).to(device)

AttributeError: module 'finetune_wav2vec2_utils' has no attribute 'AudioClassifier'

In [12]:
w2v2.train_model(model, train_loader, val_loader, device, num_epochs=10)

Epoch 1:   0%|          | 0/284 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
  File "/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
  File "/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 211, in collate
    return [
  File "/home/sirjanhansda/miniconda3/envs/FrontEra/lib/python3.10/site-packages/torch/utils/data/_utils/collate.py", line 212, in <listcomp>
    co